In [1]:
import sys
sys.path.append("..")
from model import model
from model import dataset as ds
from torch.utils.data import DataLoader
import os
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE
train_dataset = ds.ImageDataset(
    annotations_path="../datasets/annotations/",
    images_path="../../frames/",
    channels=3
)


len(train_dataset)
train_loader = DataLoader(
    train_dataset,
    batch_size=len(train_dataset.images),
    shuffle=True,
    num_workers=os.cpu_count()
    )


#TODO Read: https://d2l.ai/chapter_computer-vision/anchor.html


# Option 1 data point = {image_path, bbox_coordinates, class}
# Option 2 just use the first bbox always and ignore the rest. Not good, but for testing purposes it can suffice
# Option 3 (Probably ideal) use anchor boxes. I'm hesitant to this idea because we only have one class and it feels a bit too cumbersome in this context
model = model.BoxRegressor().to(DEVICE)
bbox_loss_func = torch.nn.MSELoss()
optimizator = torch.optim.Adam(model.parameters(), lr=.01)
train_loss = []

for epoch in range(5):
    model.train()
    loss = 0
    for (images, bboxes) in train_loader:
        (images, bboxes) = (images.to(DEVICE), bboxes.to(DEVICE))
        predictions = model(images)
        bbox_loss = bbox_loss_func(predictions, bboxes)
        loss += bbox_loss
        # ...